In [27]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
from pyngrok import ngrok

# Load your datasets
recipes_df = pd.read_csv('/content/70000_recipes_nutrients_cleaned_final.csv')
health_df = pd.read_csv('/content/health_age_data_70000_synthetic.csv')

# --- Clean & Prepare Data ---
health_df['Disease'] = health_df['Disease'].str.lower().str.strip()
health_exploded = health_df.assign(Disease=health_df['Disease'].str.split(', ')).explode('Disease')

# --- Disease Frequency ---
disease_counts = health_exploded['Disease'].value_counts().reset_index()
disease_counts.columns = ['Disease', 'Count']
fig_disease = px.bar(disease_counts, x='Disease', y='Count', title="🦠 Disease Frequency")

# --- Age Distribution ---
fig_age = px.histogram(health_df, x='Ages', nbins=30, title="👥 Age Distribution")

# --- Gender Distribution ---
fig_gender = px.pie(health_df, names='Gender', title="🚻 Gender Breakdown")

# --- BMI Distribution ---
health_df['BMI'] = health_df['Weight'] / (health_df['Height'] / 100) ** 2
fig_bmi = px.histogram(health_df, x='BMI', nbins=30, title="📊 BMI Distribution")

# --- Top Recipes ---
top_recipes = recipes_df['recipe_name'].value_counts().nlargest(10).reset_index()
top_recipes.columns = ['Recipe Name', 'Count']
fig_recipes = px.bar(top_recipes, x='Recipe Name', y='Count', title="🍽 Top 10 Recipes")

# --- Nutrient Distributions ---
fig_nutrients = []
for col in ['calories', 'protein', 'fat', 'carbohydrates']:
    fig = px.histogram(recipes_df, x=col, nbins=30, title=f"🥗 {col.title()} in Recipes")
    fig_nutrients.append(fig)

# --- Avg Nutrient Targets by Disease ---
avg_targets = health_exploded.groupby('Disease')[['Calories', 'Protein', 'Fat', 'Carbohydrates', 'Fiber']].mean().reset_index()
fig_targets = px.bar(
    avg_targets.melt(id_vars='Disease', var_name='Nutrient', value_name='Target'),
    x='Disease', y='Target', color='Nutrient', barmode='group',
    title="🍎 Avg Daily Nutrient Targets by Disease"
)

# --- Create Dashboard Layout ---
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("NutriFusion EDA Dashboard", className="text-center my-4"),

    dbc.Row([dbc.Col(dcc.Graph(figure=fig_disease))]),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig_age)), dbc.Col(dcc.Graph(figure=fig_gender))]),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig_bmi))]),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig_recipes))]),

    html.Hr(),
    html.H3("🥗 Nutrient Distributions in Recipes"),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig), width=6) for fig in fig_nutrients[:2]]),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig), width=6) for fig in fig_nutrients[2:]]),

    html.Hr(),
    dbc.Row([dbc.Col(dcc.Graph(figure=fig_targets))]),

], fluid=True)

# --- Run Dashboard in Colab using ngrok ---
port = 8050
public_url = ngrok.connect(port)
print(f"🚀 Your NutriFusion Dashboard is live here:\n{public_url}")

app.run(port=port, use_reloader=False)


🚀 Your NutriFusion Dashboard is live here:
NgrokTunnel: "https://f635036b7df1.ngrok-free.app" -> "http://localhost:8050"


<IPython.core.display.Javascript object>